In [55]:
! pip install openpyxl gensim fasttext===0.9.2 razdel nmslib xlsxwriter razdel

In [61]:
import re, os, tqdm
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
import fasttext
import razdel
import nmslib
import scipy as sp
import joblib, pickle
from itertools import combinations
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

#папка с необходимыми файлами для обучения модели
path = "/kaggle/input/hacksai-3/"
temp = "/kaggle/working/"

russian_stopwords = open(os.path.join(path, 'stopwords-ru.txt'), 'r').read().split('\n')
prod_name_clf = joblib.load(os.path.join(path,'product_group.pkl'))
anomaly_detector = CatBoostClassifier()
anomaly_detector.load_model(os.path.join(path,'anomaly_detector.model'))
reg_clf = fasttext.load_model(os.path.join(path,'reglament_predictor.model'))
ved_thes = pd.read_csv(os.path.join(path,'ved_dict.csv'), sep=';')
for code_col in ["GRUPPA", "TOV_POZ", "SUB_POZ", "VED", "RAZDEL"]:
    ved_thes.loc[ved_thes[code_col].notna(), code_col] =\
        ved_thes.loc[ved_thes[code_col].notna(), code_col].astype(int).astype(str)
with open(os.path.join(path,'ved_dict.pickle'), "rb") as handle:
    ved_dict = pickle.load(handle)
pmi_hist = pd.read_csv(os.path.join(path,'pmi_features.csv'), sep=';')
pmi_hist[pmi_hist.columns[:5]] = pmi_hist[pmi_hist.columns[:5]].astype(str)
index = nmslib.init(method='napp', space='cosinesimil')
index.loadIndex(os.path.join(path,'index_ved'), load_data=True)

indexed_data_dict = joblib.load(os.path.join(path,'index_map.pkl'))
ft_model_v2 = fasttext.load_model(os.path.join(path, 'fb_model_v2.bin'))


def delete_stopwords(s):
    return ' '.join([word for word in (re.sub(r'[()\s+]', u' ', s)).split() if word.lower() not in russian_stopwords]).split()


def delete_punctuation(s):
    symbols = [
           '\t', '!','%','&',"'",'(',')','*','+',',','-','.', '\\', '®',
           '/', '~','«','\xad','¯','°','`','±','²','³','·','º', '»', ':',';','<','=','?','@',
           'É','Ó','Ö','×','Ø','Ü','ä','é','ö','÷','İ','Š','˂','˚','̆','Ι', 'Λ', '[','\\',']','_','`',
          '\u200e','‐','–', '—', '‘', '’', '“', '”', '•', '…', '‧', '⁰', '₂', '℃', '№', '™', 
           'Ⅰ', 'Ⅱ', 'Ⅲ', 'Ⅳ', '↑', '−', '∞', '≤', '\uf0d2' '️','（', '）', '，', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'
          ]
    
    return re.sub(r'[{}\s+]'.format(''.join(symbols)), u' ', s.replace('\xad', ' '))


def tokenize_with_razdel(text):
    tokens = [token.text for token in razdel.tokenize(text)]
    
    return tokens


vectorizer = joblib.load(os.path.join(path,'vectorizer.pkl'))


def get_product_name(data):
    predicts = prod_name_clf.predict(vectors)
    return predicts
    
    
def get_ved(data):    
    model_ved1 = joblib.load(os.path.join(path,'ved1.pkl'))
    model_ved2 = joblib.load(os.path.join(path,'ved2.pkl'))
    model_ved3 = joblib.load(os.path.join(path,'ved3.pkl'))
    
    ved1 = model_ved1.predict(data)
    data_ved2 = np.hstack([data, ved1])
    ved2 = model_ved2.predict(data_ved2)
    
    del data_ved2
    gc.collect()
    
    data_ved3 = np.hstack([data, ved1, ved2])
    ved3 = model_ved3.predict(data_ved3)
    
    return str(ved1)+str(ved2)+str(ved3)


def write_file(data):

    list1 = data[['Номер продукции', 'Общее наименование продукции',
             'Коды ТН ВЭД ЕАЭС', 'Технические регламенты',
             'Группа продукции', 'Наличие ошибки']]
    list1.columns = ['Код', 'Общее наименование продукции',
            'ТН ВЭД ЕАЭС', 'Технические регламенты',
            'Группа продукции', 'Наличие ошибки']
    
    list2 = data[['Номер продукции', 'Общее наименование продукции',
      'Коды ТН ВЭД ЕАЭС_predicted', 'Технические регламенты_predicted',
     'Группа продукции_predicted']]
    list2.columns = ['Код', 'Общее наименование продукции',
                    'ТН ВЭД ЕАЭС', 'Технические регламенты',
                    'Группа продукции']
    with pd.ExcelWriter('JETFORK_Тесты2.xlsx', engine='xlsxwriter') as writer:
        list1.to_excel(excel_writer = writer, index = False, sheet_name='Тест1')

        list2.to_excel(excel_writer = writer, index = False, sheet_name='Тест2')
    
    
def pmi_predict(dataframe, x, y):
    """Calculate PMI for income data based on historical stats.
    
    Args:
        dataframe_hist - historical PMI stats;
        dataframe - new data to calculation;
        x - column left;
        y - column right.
    Returns:
        (pd.DataFrame)
    """
    dataframe = dataframe.merge(
        pmi_hist[[x, y, f"pmi_{x}/{y}"]].drop_duplicates(), 
        on=[x, y], how="left"
    ).fillna({f"pmi_{x}/{y}": -5})
    return dataframe


def add_ved_info(dataframe):
    """ Add VED main categories.
    
    Args:
        dataframe - income data.
    Returns:
        (pd.DataFrame)
    """
    return dataframe.merge(
        ved_thes[["VED", "RAZDEL", "GRUPPA", "TOV_POZ"]].drop_duplicates() \
            .rename(columns={"VED": "Коды ТН ВЭД ЕАЭС"}) \
            .astype(str),
        on=["Коды ТН ВЭД ЕАЭС"], how="left"
    ).fillna("-1")


IDXS = ["Номер продукции"]
FEATURES = ["Технические регламенты", "Группа продукции", "RAZDEL", "GRUPPA", "TOV_POZ"]
def predict_anomaly(dataframe):
    """ Detect outliers in dataframe attributes.
    
    Args:
        dataframe - income data.
    Returns:
        (pd.DataFrame)
    """
    d = dataframe.copy()
    
    dataframe = dataframe.dropna(subset=["Коды ТН ВЭД ЕАЭС", "Технические регламенты", "Группа продукции"])
    dataframe["Коды ТН ВЭД ЕАЭС"] = dataframe["Коды ТН ВЭД ЕАЭС"].astype(str) \
        .str.split("; ") \
        .apply(set)
    dataframe["Технические регламенты"] = dataframe["Технические регламенты"].astype(str) \
        .str.split("; ") \
        .apply(set)
    dataframe["Группа продукции"] = dataframe["Группа продукции"].astype(str) \
        .str.split("; ") \
        .apply(set)
    dataframe = dataframe[~(dataframe["Коды ТН ВЭД ЕАЭС"].apply(len) > 6)].reset_index(drop=True)
    dataframe = dataframe[~(dataframe["Группа продукции"].apply(len) > 2)].reset_index(drop=True)

    dataframe = dataframe.explode("Коды ТН ВЭД ЕАЭС") \
        .dropna() \
        .explode("Технические регламенты") \
        .dropna() \
        .explode("Группа продукции") \
        .dropna() \
        .reset_index(drop=True)

    dataframe = add_ved_info(dataframe)
    dataframe = dataframe[IDXS + FEATURES]
    
    for f_1, f_2 in combinations(FEATURES, 2):
        if f_1 != f_2:
            dataframe = pmi_predict(dataframe, f_1, f_2)
    
    dataframe["outlier"] = anomaly_detector.predict(dataframe[anomaly_detector.feature_names_])
    d = d.merge(dataframe.groupby(IDXS)["outlier"].max().reset_index(), on=IDXS, how="left")["outlier"].fillna(1).astype(int)
    return d


def predict_reg(s):
    """ Predict tech reg based on product name
    
    Args:
        s - input string.
    Returns:
        (str)
    """
    s = ' '.join(delete_stopwords(delete_punctuation(s)))
    label = reg_clf.predict(s)[0]
    return ved_dict.get(label[0][9:].replace('_', ' '))


def get_ved(vectors_ft):
    neighbours = index.knnQueryBatch(vectors_ft, k=1, num_threads=10)
    data['index'] = np.array(neighbours)[:, 0].reshape(-1)
    # data['distance'] = np.array(neighbours)[:, 1].reshape(-1)
    veds = data['index'].map(indexed_data_dict)
    return veds.apply(lambda x: ''.join(x))

In [57]:
data = pd.read_csv(os.path.join(path, 'outlier_whole_preds.csv'), sep=';').drop('outlier', axis=1)

data['clean_product_name'] = data['Общее наименование продукции'].fillna('') \
    .str.lower() \
    .apply(lambda x: ' '.join(delete_stopwords(delete_punctuation((x)))))
vectors = vectorizer.transform(data['clean_product_name'])
vectors_ft  = np.array([ft_model_v2.get_sentence_vector(text) for text in data['clean_product_name']])


In [58]:
%%time
data['Наличие ошибки'] = predict_anomaly(data)
print(1)
data['Группа продукции_predicted'] = get_product_name(vectors) 
print(2)
data['Технические регламенты_predicted'] = data["Общее наименование продукции"].fillna("").apply(predict_reg)
print(3)
data['Коды ТН ВЭД ЕАЭС_predicted'] = get_ved(vectors_ft)

In [62]:
write_file(data)